#### Import field data

In [2]:
from initialisation import *

Vs, Ts, FC_star = {}, {}, {}

# Import empirical vegetation density
Vs['grass'] = matrix_to_dict(np.loadtxt('field-data/grassland/vegetation.asc', skiprows=6)[1:-1,1:-1])
Vs['shrub'] = matrix_to_dict(np.loadtxt('field-data/shrubland/vegetation.asc', skiprows=6)[1:-1,1:-1])

# Import empirical topography
Ts['grass'] = matrix_to_dict(np.loadtxt('field-data/grassland/topography.asc', skiprows=6)[1:-1,1:-1])
Ts['shrub'] = matrix_to_dict(np.loadtxt('field-data/shrubland/topography.asc', skiprows=6)[1:-1,1:-1])

# Import MAHLERAN sediment functional connectivity
FC_star['grass'] = matrix_to_dict(np.loadtxt('MAHLERAN-data/grassland/sediment-connectivity.asc', skiprows=6)[1:-1,1:-1])
FC_star['shrub'] = matrix_to_dict(np.loadtxt('MAHLERAN-data/shrubland/sediment-connectivity.asc', skiprows=6)[1:-1,1:-1])


#### Initialise lattice

In [8]:
# Create baseline 2D lattice, empirical field data default width=20 and height=60
G, bulk_nodes = generate_baseline_lattice(20,60)

# Adapt G to shrubland empirical data, assuming deterministic dynamics
G_deter = adapt_lattice_to_topography(G, Ts['shrub'], 'deterministic')

# Initialise simulation
S, current, branches, new_active, size = declare_simulation_variables(G_deter)
print('Computing node coupling dictionary...')
#C = create_node_coupling_dictionary(G_deter)
slope_per_edge = compute_slope_differences(G_deter, Ts['shrub'])
print('Computing SC...')
print(type(G))
SC = compute_SC(G, slope_per_edge)


Computing node coupling dictionary...
Computing SC...
<class 'networkx.classes.digraph.DiGraph'>


TypeError: tuple indices must be integers or slices, not tuple

#### Simulation run

#### Avalanche size analysis